Matthew Sett
<br>
Date: Feb. 3, 2022
<br>
PHYS 2030 W23

# <center><font color=#46769B>Exercise 11: Data fitting</font></center>

## <font color=#46769B>Introduction</font>

Monte Carlo sampling is a really useful tool for fitting data. Let's illustrate the idea with a few simple examples.

<u>Example 1:</u> Suppose you have several different measurements for a parameter $x$, e.g., 

$$x = 3.8 \pm 0.3, \; 3.4 \pm 2.2 , \; 5.0 \pm 1.2 , \;  4.2 \pm 0.1 \, , \;  {\rm etc.}$$

How do you combine these different measurements to find the best value for $x$ (and its uncertainty)? Note that the uncertainties are different and the best value of $x$ is not simply the mean of all these measurements. Intuitively, we expect that measurements with smaller uncertainties should carry more weight when determining the best value for $x$, $x$ is probably closer to $3.8$ and $4.2$ compared to the other values.

<u>Example 2:</u> Suppose we have an object in uniform motion, with position as a function of time given by 

$$x(t) = v t \, ,$$

where $v$ is the velocity and the initial position at $t=0$ is $x(0) = 0$. Now suppose $x$ measurements are positions *at different times* $t_1, t_2, {\rm etc}$, e.g.,

$$x(t_1) = 0.8 \pm 0.3, \; x(t_2) = 1.8 \pm 0.8 , \; x(t_3) = 2.0 \pm 0.2 , \;  x(t_4) = 2.5 \pm 0.5 \, ,  \; {\rm etc.}$$

How do we use the data to determine $v$ (and its uncertainties)?

Problems of these types are known as __inverse problems__. In a "non-inverse" problem, we have a PDF that we use to calculate something (an observable). Here, we are going in the opposite direction: using the observables (measured by observations) to determine the PDF for the parameters that determine the observables.

Another name for all this is __model fitting__ or __fitting a model to data__. The idea is:

- Write down a model to describe the data
- The model depends one or more unknown parameters
- Sample the parameters with a Monte Carlo simulation
- Use the data to determine which samples are good or bad fits

In the second example, the model was *uniform motion*, described by the function $x(t) = v t$. The unknown parameter is $v$, which we will show how to fit using data. In the first example, the model is trivial: it is just *a constant* and that constant is the parameter to be fit.

## <font color=#46769B>Monte Carlo sampling</font>

We will go through how to use Monte Carlo sampling (and importance sampling specifically) to answer the questions raised here, step-by-step. 

<u>Example 1: Finding the best value.</u> The steps are:

1. Generate $N$ samples for $x$ from a proposal distribution $Q(x)$ of your choice.

2. Define the target distribution $P(x)$ for your samples. $P(x)$ is where we compare $x$ to data (the different measurements for $x$). Large $P(x)$ means $x$ fits the data well, and small $P(x)$ means $x$ does not.

3. Calculate the weights $w_i = P(x_i)/Q(x_i)$ for your samples $x_i$. (In practice, this formula will be modified, discussed below.)

4. The "best value" of $x$ is the weighted mean
$$\langle x \rangle = \frac{1}{N} \sum_{i=0}^{N-1} w_i x_i$$
The uncertainty is the weighted standard deviation $\Delta x$, computed as the square root of the weighted variance
$$\Delta x^2 = \langle x^2 \rangle - \langle x \rangle^2$$
where $\langle x^2 \rangle = \sum_i w_i x_i^2$.

Lastly, we need to specify what $P(x)$ is. We will denote the different measurements for $x$ in terms of their central values $\mu_a$ and uncertainties $\sigma_a$, i.e., 

$$x = \mu_1 \pm \sigma_1 , \; \mu_2 \pm \sigma_2 , \; \mu_3 \pm \sigma_3 , \; {\rm etc.}$$

where index $a = 1,2,3, {\rm etc}$ labels the different measurements. Here we calculate the $\chi^2$ as a function of $x$:

$$\chi^2(x) = \frac{(x - \mu_1)^2}{\sigma_1^2} + \frac{(x - \mu_2)^2}{\sigma_2^2} + \frac{(x - \mu_3)^2}{\sigma_3^2} + \dots
= \sum_a \frac{(x - \mu_a)^2}{\sigma_a^2} \qquad (1) $$ 

Note that all the $\mu_a$ and $\sigma_a$ are just numbers that come from the data, and so $\chi^2(x)$ is just a function of our unknown $x$. The probability density is related to the $\chi^2$ by

$$P(x) \propto e^{-\tfrac{1}{2} \chi^2(x)}$$

A few comments:
- There are a few assumptions implicit defining $P(x)$ in terms of the $\chi^2$, which we won't discuss much. The most important one is that the measurements of $x$ are all *independent* from one another. If everyone is measuring $x$ with the same biased apparatus, that would not be independent.

- We don't know the overall constant for $P(x)$, which is fixed by requiring $\int dx \, P(x) = 1$ when integrated over all $x$ values.

To deal with the second point, we will consider the *unnormalized* PDF 

$$P(x) = e^{-\tfrac{1}{2} \chi^2(x)} \, , \qquad (2)$$

such that $\int dx \, P(x) \ne 1$. Now we need to change our formula for the weights so that the incorrect normalization factor for $P(x)$ cancels out:

$$w_i = \frac{ P(x_i)/Q(x_i) }{\frac{1}{N} \sum_j P(x_j)/Q(x_j) } \qquad (3) $$

Recall that if we had normalized $P(x)$ properly the term in the denominator would be (approximately) 1, and we would get back the usual formula. The following lines of code are what you need to do this:

```py
w_unnormalized = P(x)/Q(x)
w = w_unnormalized/np.mean(w_unnormalized)
```

<u>Example 2: Fitting a model to data.</u> We have a model $x(t) = v t$. We have our data points for $x$:

$$x(t_1) = \mu_1 \pm \sigma_1 , \; x(t_2) = \mu_2 \pm \sigma_2 , \; x(t_3) = \mu_3 \pm \sigma_3 , \; {\rm etc.}$$

where $\mu_a$ is central value, $\sigma_a$ is uncertainty, and now we have another quantity $t_a$ that is the *time* for each data point. 

1. Generate $N$ samples for $v$ from a proposal distribution $Q(v)$ of your choice.

2. Evaluate the target distribution $P(v)$ for your samples. Now the $\chi^2$ function depends on $v$ (since all $\mu_a, \sigma_a, t_a$ are just numbers):

$$\chi^2(v) = \frac{(x(t_1) - \mu_1)^2}{\sigma_1^2} + \frac{(x(t_2) - \mu_2)^2}{\sigma_2^2} + \frac{(x(t_3) - \mu_3)^2}{\sigma_3^2} + \dots
= \sum_a \frac{(x(t_a) - \mu_a)^2}{\sigma_a^2} 
= \sum_a \frac{(v t_a - \mu_a)^2}{\sigma_a^2} \qquad (4) $$ 

3. Calculate the (normalized) weights

$$w_i = \frac{ P(v_i)/Q(v_i) }{\frac{1}{N} \sum_j P(v_j)/Q(v_j) } \qquad (5) $$

4. The "best value" of $v$ is the weighted mean $\langle v \rangle$ and the uncertainty is the weighted standard deviation $\Delta v$.

Our goals for this notebook are:
- Use importance sampling to fit parameters from data.

Required reading:
- *Lesson 4: Importance sampling*



## <font color=#46769B>Part (a)</font>

Suppose we have 10 different (independent) measurements of the gravitational acceleration constant $g$. These are (in ${\rm m/s^2}$):

$$9.74 \pm 0.12,  \; 9.44 \pm 0.30,  \; 9.68 \pm 0.10, \; 10.02 \pm 0.38, \; 9.84 \pm 0.36, \;
        9.88 \pm 0.25,  \; 9.24 \pm 0.50,  \; 9.99 \pm 0.15,  \; 9.58 \pm 0.40,  \; 9.45 \pm 0.40 $$
        
We will denote $\mu_g$ as all the central values and $\sigma_g$ as all the uncertainties listed above. 

Your tasks are as follows:
- Following the logic described above, use Monte Carlo sampling to combine these measurements into a single central value and uncertainty. Take $N = 10^5$ samples for $g$.

- Make a (weighted) histogram plot of your samples for $g$.

Note: You will need to sample $g$ from a proposal distribution $Q(g)$. Choose a normal distribution

$$Q(g) = \mathcal{N}(\mu_Q,\sigma_Q) $$

where $\mu_Q$ and $\sigma_Q$ here are numbers that *you pick* to get good results. They are *not* the same as the $\mu_g$ and $\sigma_g$, which are fixed arrays corresponding to the data.




In [33]:
import numpy as np
import matplotlib.pyplot as plt

# Data is here (central values and uncertainties)
mu_g = np.array([9.74, 9.44, 9.68, 10.02, 9.84, 9.88, 9.24, 9.99, 9.58, 9.45])
sigma_g = np.array([0.12, 0.30, 0.10, 0.38, 0.36, 0.25, 0.50, 0.15, 0.40, 0.40])
sum_mu_g = np.sum(mu_g)
sum_sigma_g = np.sum(sigma_g)
N = 10**5
g = np.random.normal(9.8,0.8,N)
x = np.linspace(0,100,N)
# Your code here
#the chi squared values
def chisq(x):
  chi = []
  for i in range(N):
    chi.append(np.sum((x[i]-mu_g)**2/sigma_g**2))
  print(chi)
chisq(g)

#plotting p(x)

def PX(chisq = x):
  np.exp(-0.5*x)

w = PX(g)/g



[36.05227293990523, 170.99217918938686, 14.04792289373448, 231.52205475684653, 86.09941217430978, 123.23635444408896, 11.378673077514557, 12.642923808685415, 113.25056692158726, 418.94598112414775, 12.36880712391628, 105.91890065949437, 105.61965386319069, 439.23241399395846, 381.4724147065639, 9.343402466641383, 72.38034377815353, 6.8994340996708825, 7.908804890319675, 218.90054217358156, 425.97075087655537, 515.9389588899733, 226.38074330263365, 188.96001013139872, 21.955045684055246, 174.08672139116294, 67.22500990248854, 10.856302138184843, 189.9338589628372, 24.98423488535204, 71.32883899580453, 6.92734479409032, 101.44500861085724, 7.115130906315975, 12.886702080083596, 9.869208537721828, 157.05374258182496, 14.38908851715573, 7.0114072193916135, 102.52712682282029, 101.08211423215336, 46.871857064872344, 13.671575278377189, 10.112355904151563, 7.297758001239337, 17.657159006469925, 91.76419950286063, 42.281914074780644, 34.718774422339486, 9.158574154068376, 194.0126095135227, 1

TypeError: ignored

## <font color=#46769B>Part (b)</font>

Consider an object that starts at rest and moves with a uniform acceleration due to gravity. Let $y(t)$ denote the distance fallen in time $t$.

Suppose we have the results of 5 experiments for $y(t)$ (in meters) at 5 different times

$$y(t_1) = 4.7 \pm 0.7, \; y(t_2) = 17.7 \pm 1.5, \; y(t_3) = 45.5 \pm 3.0, \; y(t_4) = 75.8 \pm 9.0 , \; 
y(t_5) = 117.4 \pm 10.0 \; .$$

where the times are (in seconds)

$$t_1 = 1 , \; t_2 = 2, \; t_3 = 3 , \; t_4 = 4 , \; t_5 = 5 \; .$$ 

Your tasks are as follows:

- Following the logic described above, use importance sampling to fit the model $y(t) = \frac{1}{2} g t^2$ to the data to determine $g$ (quote the weighted mean value as the best fit value and standard deviation as the error). Take $N = 10^5$ samples for $g$.

- Make a (weighted) histogram plot of your samples for $g$.

Similar to Part (a), consider a normal distribution for sampling $g$.

In [ ]:
# Data
mu_y = np.array([4.7, 17.7, 45.5, 75.8, 117.4])
sigma_y = np.array([0.7, 1.5, 3.0, 9.0, 10.0])
t = np.array([1, 2, 3, 4, 5])

# Your code here
